In [ ]:
from GarbageDataset import GarbageDataset, collate_fn
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor, YolosModel, YolosForObjectDetection, YolosConfig
import torch

In [ ]:
# Test model
dataset = GarbageDataset('hustvl/yolos-small', 'dataset/dataset_10000_update_0912.xml', 'dataset/參賽者訓練資料集下載(10000)/', (416, 416))
dataloader = DataLoader(dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)
model = YolosForObjectDetection.from_pretrained('hustvl/yolos-small')
for inputs, labels in dataloader:
    print(inputs.shape)
    print(labels)
    outputs = model(inputs, labels = labels)
    print(outputs.loss)
    break

In [ ]:
def label_to_device(labels, device):
    for label in labels:
        for key in label:
            label[key] = label[key].to(device)
    return labels

In [ ]:
# Training Config
model_name = 'hustvl/yolos-small'


EPOCHS = 10

batch_size = 4
learning_rate = 0.001
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

image_size = (416, 416)

In [ ]:
# Training
dataset = GarbageDataset(model_name, 'dataset/dataset_10000_update_0912.xml', 'dataset/參賽者訓練資料集下載(10000)/', image_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
model = YolosForObjectDetection(YolosConfig.from_pretrained(model_name))
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


model.to(device)
for epoch in range(EPOCHS):
    log_loss = 0
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = label_to_device(labels, device)
        outputs = model(inputs, labels = labels)
        optimizer.zero_grad()
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        log_loss += loss.item()
        print(loss.item())
    print(f'Epoch: {epoch}, Loss: {log_loss/len(dataloader)}')
    